In [1]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi

In [2]:
with open('../data/wikipedia_documents.json', 'r', encoding='utf-8') as f:
    wiki_data = json.load(f)

documents = [v['text'] for v in wiki_data.values()]

In [3]:
file_path = 'retrieve_result.csv' 
df = pd.read_csv(file_path)

bm25_failures = df[df['bm25'] == '20+']

failed_queries = bm25_failures['question'].tolist()

In [8]:
vectorizer = TfidfVectorizer()
vectorizer.fit(documents)

tokenized_corpus = [doc.split() for doc in documents]
bm25 = BM25Okapi(tokenized_corpus)

In [10]:
output_data = []

for query in failed_queries:
    # IDF 계산
    tokens = vectorizer.transform([query])
    idf_values = {word: vectorizer.idf_[vectorizer.vocabulary_[word]] 
                  for word in query.split() if word in vectorizer.vocabulary_}
    
    # BM25 상위 20개 문서 번호 추출
    tokenized_query = query.split()  
    doc_scores = bm25.get_scores(tokenized_query) 
    top_n_indices = doc_scores.argsort()[::-1][:20]  
    top_n_doc_ids = top_n_indices.tolist()

    # 데이터 저장
    output_data.append({
        'question': query,
        'idf_values': idf_values,
        'bm25_top_20': top_n_doc_ids
    })

In [11]:
output_df = pd.DataFrame(output_data)
output_df.to_csv('bm25_failure_analysis.csv', index=False)

print("BM25 실패 데이터 분석 결과가 'bm25_failure_analysis.csv'에 저장되었습니다.")

BM25 실패 데이터 분석 결과가 'bm25_failure_analysis.csv'에 저장되었습니다.


## 개선

### 1. 한국어 토크나이저

#### (1) Konply - okt

In [9]:
from konlpy.tag import Hannanum, Kkma, Komoran, Okt #mecab 설치 오류
from datasets import load_from_disk

hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
okt = Okt()

In [14]:
train_dataset = load_from_disk('../data/train_dataset')
train_split = train_dataset['train']

questions = train_split['question'][31:32]

In [15]:
def remove_josa_hannanum(sentence):
    tokens = hannanum.pos(sentence)
    filtered_tokens = [word for word, pos in tokens if 'J' not in pos]  # 조사는 'J'로 시작하는 태그
    return ' '.join(filtered_tokens)

def remove_josa_kkma(sentence):
    tokens = kkma.pos(sentence)
    filtered_tokens = [word for word, pos in tokens if 'J' not in pos]  # 조사는 'J'로 시작하는 태그
    return ' '.join(filtered_tokens)

def remove_josa_komoran(sentence):
    tokens = komoran.pos(sentence)
    filtered_tokens = [word for word, pos in tokens if 'J' not in pos]  # 조사는 'J'로 시작하는 태그
    return ' '.join(filtered_tokens)

def remove_josa_okt(sentence):
    tokens = okt.pos(sentence)
    filtered_tokens = [word for word, pos in tokens if pos != 'Josa']  # Okt에서 조사 제거
    return ' '.join(filtered_tokens)

In [16]:
for i, question in enumerate(questions):
    print(f"질문 {i+1} 원본: {question}")
    
    print(f"  Hannanum 조사 제거 후: {remove_josa_hannanum(question)}")
    print(f"  Kkma 조사 제거 후: {remove_josa_kkma(question)}")
    print(f"  Komoran 조사 제거 후: {remove_josa_komoran(question)}")
    print(f"  Okt 조사 제거 후: {remove_josa_okt(question)}\n")

질문 1 원본: 도연주와 백승희는 무슨 사이로 알려졌나?
  Hannanum 조사 제거 후: 도연주 백승희는 무슨 사이 알리 어 지 었나 ?
  Kkma 조사 제거 후: 도 연주 백 승희 무슨 사이 알리 어 지 었 나 ?
  Komoran 조사 제거 후: 도 연주 백승희 무슨 사이 알리 어 지 었 나 ?
  Okt 조사 제거 후: 도 연주 백승희 무슨 사이 알려졌나 ?



In [24]:
import time
from konlpy.tag import Okt
import json

# Okt 형태소 분석기 초기화
okt = Okt()

# wikipedia_documents.json 파일 로드
with open('../data/wikipedia_documents.json', 'r', encoding='utf-8') as f:
    wiki_data = json.load(f)

# 첫 번째 문서 텍스트 추출
first_document = list(wiki_data.values())[0]['text']

# 시간 측정 시작
start_time = time.time()

# 첫 번째 문서에 대해 Okt 토크나이징 수행
tokenized_document = okt.morphs(first_document)

# 시간 측정 종료
end_time = time.time()

# 처리 시간 계산
processing_time = end_time - start_time

# 결과 출력
print(f"첫 번째 문서 토크나이징에 걸린 시간: {processing_time:.4f}초")

첫 번째 문서 토크나이징에 걸린 시간: 0.0779초


In [25]:
from transformers import ElectraTokenizer
from datasets import load_from_disk

# monologg/koelectra-base-v3-finetuned-korquad 토크나이저 로드
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-finetuned-korquad")

# Train dataset 로드
train_dataset = load_from_disk('../data/train_dataset')
train_split = train_dataset['train']

# 질문 데이터 (앞의 5개만)
questions = train_split['question'][:5]

# 질문을 토크나이징
tokenized_questions = [tokenizer(question) for question in questions]

# 토크나이징 결과 확인
for i, (question, tokenized) in enumerate(zip(questions, tokenized_questions)):
    print(f"Original Question {i+1}: {question}")
    print(f"Tokenized Question {i+1}: {tokenized}\n")

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/591 [00:00<?, ?B/s]

Original Question 1: 대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?
Tokenized Question 1: {'input_ids': [2, 6283, 4292, 6469, 4283, 6257, 4234, 10974, 10695, 4046, 4292, 2022, 4034, 6325, 6468, 4112, 35, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Original Question 2: 현대적 인사조직관리의 시발점이 된 책은?
Tokenized Question 2: {'input_ids': [2, 6533, 4199, 6596, 4084, 4497, 10425, 4234, 28316, 4007, 2412, 3432, 4112, 35, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Original Question 3: 강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?
Tokenized Question 3: {'input_ids': [2, 25812, 4106, 4070, 20358, 4056, 4556, 4073, 3065, 2129, 4112, 6776, 4110, 6237, 13414, 4753, 4034, 4070, 35, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,